In [6]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, SeamlessM4Tv2Model
from llama_cpp import Llama
from langdetect import detect
from gtts import gTTS
import io
from pydub import AudioSegment
import numpy as np
import pandas as pd
import time
import os

In [7]:
input_text = [
    ["English", "Spread out and search for heat signatures."],
    ["French", "Répartissez-vous et recherchez des signatures thermiques."],
    ["German", "Verteilen Sie sich und suchen Sie nach Wärmesignaturen."],
    ["Spanish", "Distribúyanse y busquen firmas de calor."],
    ["Italian", "Sparpagliatevi e cercate firme di calore."],
    ["Dutch", "Spreid uit en zoek naar warmtehandtekeningen."],
    ["Polish", "Rozproszyć się i szukać sygnatur cieplnych."],
    ["Portuguese", "Espalhem-se e procurem por assinaturas de calor."],
    ["Swedish", "Sprid ut er och sök efter värmesignaturer."],

    ["English", "Mark the location of survivors and form a path to guide rescue workers."],
    ["French", "Marquez l'emplacement des survivants et formez un chemin pour guider les secours."],
    ["German", "Markieren Sie den Standort der Überlebenden und bilden Sie einen Weg, um Rettungskräfte zu leiten."],
    ["Spanish", "Marquen la ubicación de los supervivientes y formen un camino para guiar a los rescatistas."],
    ["Italian", "Segnate la positione dei sopravvissuti e formate un percorso per guidare i soccorritori."],
    ["Dutch", "Markeer de locatie van overlevenden en vorm een pad om reddingswerkers te begeleiden."],
    ["Polish", "Oznacz miejsce przebywania ocalałych i wytycz ścieżkę, aby poprowadzić ratowników."],
    ["Portuguese", "Marque a localização dos sobreviventes e forme um caminho para guiar os socorristas."],
    ["Swedish", "Markera platsen för överlevande och skapa en väg för att guida räddningsarbetare."],

    ["English", "Scan the field for dry or unhealthy plants."],
    ["French", "Scannez le champ à la recherche de plantes sèches ou malades."],
    ["German", "Scannen Sie das Feld nach trockenen oder kranken Pflanzen."],
    ["Spanish", "Escaneen el campo en busca de plantas secas o enfermas."],
    ["Italian", "Scansionate il campo alla ricerca di piante secche o malsane."],
    ["Dutch", "Scan het veld op droge of ongezonde planten."],
    ["Polish", "Zeskanuj pole w poszukiwaniu suchych lub chorych roślin."],
    ["Portuguese", "Escaneie o campo em busca de plantas secas ou doentes."],
    ["Swedish", "Skanna fältet efter torra eller ohälsosamma växter."],

    ["English", "Cluster around unhealthy plants and emit a signal for manual inspection."],
    ["French", "Regroupez-vous autour des plantes malades et émettez un signal pour une inspection manuelle."],
    ["German", "Gruppieren Sie sich um kranke Pflanzen und senden Sie ein Signal für eine manuelle Inspektion."],
    ["Spanish", "Agrúpense alrededor de las plantas enfermas y emitan una señal para una inspección manual."],
    ["Italian", "Raggruppatevi intorno alle piante malsane ed emettete un segnale per un'ispezione manuale."],
    ["Dutch", "Groep rond ongezonde planten en zend een signaal voor handmatige inspectie."],
    ["Polish", "Skup się wokół chorych roślin i emituj sygnał do ręcznej inspekcji."],
    ["Portuguese", "Agrupe-se ao redor das plantas doentes e emita um sinal para inspeção manual."],
    ["Swedish", "Samlas runt ohälsosamma växter och sänd en signal för manuell inspektion."],

    ["English", "Search for plastic waste and collect it in one location."],
    ["French", "Recherchez les déchets plastiques et rassemblez-les en un seul endroit."],
    ["German", "Suchen Sie nach Plastikmüll und sammeln Sie ihn an einem Ort."],
    ["Spanish", "Busquen residuos plásticos y júntenlos en un solo lugar."],
    ["Italian", "Cercate i rifiuti di plastica e raccoglieteli in un unico posto."],
    ["Dutch", "Zoek naar plastic afval en verzamel het op één locatie."],
    ["Polish", "Szukaj plastikowych odpadów i zbierz je w jednym miejscu."],
    ["Portuguese", "Procure por resíduos plásticos e reúna-os em um único local."],
    ["Swedish", "Sök efter plastavfall och samla det på en plats."],

    ["English", "After cleaning, form a perimeter around the park."],
    ["French", "Après le nettoyage, formez un périmètre autour du parc."],
    ["German", "Bildet nach der Reinigung einen Perimeter um den Park."],
    ["Spanish", "Después de limpiar, formen un perímetro alrededor del parque."],
    ["Italian", "Dopo la pulizia, formate un perimetro attorno al parco."],
    ["Dutch", "Vorm na het schoonmaken een perimeter rond het park."],
    ["Polish", "Po sprzątaniu utwórz obwód wokół parku."],
    ["Portuguese", "Após a limpeza, forme um perímetro ao redor do parque."],
    ["Swedish", "Efter rengöring, skapa en perimeter runt parken."],

    ["English", "Form a foundation grid at these coordinates."],
    ["French", "Formez une grille de fondation à ces coordonnées."],
    ["German", "Bildet ein Fundamentraster an diesen Koordinaten."],
    ["Spanish", "Formen una cuadrícula de base en estas coordenadas."],
    ["Italian", "Formate una griglia di base a queste coordinate."],
    ["Dutch", "Vorm een funderingsrooster op deze coördinaten."],
    ["Polish", "Utwórz siatkę fundamentów w tych współrzędnych."],
    ["Portuguese", "Forme uma grade de fundação nestas coordenadas."],
    ["Swedish", "Skapa ett grundläggningsrutnät vid dessa koordinater."],

    ["English", "Illuminate the site from above."],
    ["French", "Illuminez le site depuis les airs."],
    ["German", "Erleuchten Sie die Stelle von oben."],
    ["Spanish", "Iluminen el sitio desde arriba."],
    ["Italian", "Illumina il sito dall'alto."],
    ["Dutch", "Verlicht de locatie van bovenaf."],
    ["Polish", "Oświetl miejsce z góry."],
    ["Portuguese", "Ilumine o local de cima."],
    ["Swedish", "Belys platsen uppifrån."],

    ["English", "Form a large circle and flash lights in sync with the music."],
    ["French", "Formez un grand cercle et clignotez les lumières en rythme avec la musique."],
    ["German", "Bilden Sie einen großen Kreis und blinken Sie die Lichter im Takt der Musik."],
    ["Spanish", "Formen un gran círculo y parpadeen las luces al ritmo de la música."],
    ["Italian", "Formate un grande cerchio e lampeggiate le luci a ritmo di musica."],
    ["Dutch", "Vorm een grote cirkel en knipper met de lichten in sync met de muziek."],
    ["Polish", "Utwórz duży okrąg i migaj światłami w rytm muzyki."],
    ["Portuguese", "Forme um grande círculo e pisque as luzes em sincronia com a música."],
    ["Swedish", "Skapa en stor cirkel och blinka med lamporna i takt med musiken."],

    ["English", "Morph into a star shape and rotate clockwise."],
    ["French", "Formez une étoile et tournez dans le sens des aiguilles d'une montre."],
    ["German", "Bilden Sie einen Stern und drehen Sie sich im Uhrzeigersinn."],
    ["Spanish", "Formen una estrella y giren en el sentido de las agujas del reloj."],
    ["Italian", "Formate una stella e ruotate in senso orario."],
    ["Dutch", "Vorm een ster en draai met de klok mee."],
    ["Polish", "Utwórz gwiazdę i obróć zgodnie z ruchem wskazówek zegara."],
    ["Portuguese", "Forme uma estrela e gire no sentido horário."],
    ["Swedish", "Bild en stjärna och rotera medsols."]
]

In [8]:
# Function to update results
def update_results(results_df, output_path, model, language, input_text, translated_text, model_time, time):
    # Create a new DataFrame if results_df is None
    if results_df is None:
        results_df = pd.DataFrame(columns=["Model", "Language", "Input", "Output", "Model time", "Time (s)"])
    
    # Create a new DataFrame for the current entry
    new_entry = pd.DataFrame([{
        "Model": model,
        "Language": language,
        "Input": input_text,
        "Output": translated_text,
        "Model time": model_time,
        "Time (s)": time,
    }])

    # Concatenate the new entry with the existing DataFrame
    results_df = pd.concat([results_df, new_entry], ignore_index=True)

    # Save the DataFrame to the specified output path
    results_df.to_csv(output_path, index=False)
    return results_df



### speech processing models

In [28]:

# Setup device and model
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-medium"
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True).to(device)
processor = AutoProcessor.from_pretrained(model_id)

# Function to translate audio using Whisper
def translate_audio(audio_file):
    try:
        if isinstance(audio_file, io.BytesIO):
            audio_file.seek(0)
            audio = AudioSegment.from_file(audio_file, format="mp3")
            audio = audio.set_frame_rate(16000).set_channels(1)
            audio_array = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
            audio_file = {"array": audio_array, "sampling_rate": 16000}
        elif isinstance(audio_file, str):
            pass
        else:
            return None, None  # Ensure consistent return

        start_time = time.time()
        input_features = processor(audio_file["array"], sampling_rate=audio_file["sampling_rate"], return_tensors="pt").input_features
        input_features = input_features.to(device)
        forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="translate")
        generated_tokens = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
        end_time = time.time()
        translated_text = processor.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translated_text, end_time - start_time
    except Exception as e:
        return f"Error during audio translation: {e}", None

# Function to process text using Whisper
def process_text_whisper(text):
    try:
        language = detect(text)
        tts = gTTS(text=text, lang=language)
        audio_buffer = io.BytesIO()
        tts.write_to_fp(audio_buffer)
        audio_buffer.seek(0)
        translated_text, model_time = translate_audio(audio_buffer)
        return translated_text, model_time
    except Exception as e:
        return f"Error: {e}", None

In [ ]:
# Example Usage
output_file = "whisper-medium-results.csv"
existing_df = pd.read_csv(output_file) if os.path.exists(output_file) else None

for language, text in input_text:
    start_time = time.time()
    translated_text, model_time = process_text_whisper(str(text))
    end_time = time.time()

    # Append a new entry to the results DataFrame
    existing_df = update_results(  # Update `existing_df` directly
        results_df=existing_df,
        output_path=output_file,
        model="whisper-medium",
        language=language,
        input_text=text,
        translated_text=translated_text,
        model_time=model_time,
        time=end_time - start_time 
    )

print("whisper-medium done")
    


In [ ]:
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

def seamless_model(audio_file):
    try:
        # Set the device (use GPU if available)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Reset audio file pointer and load audio
        audio_file.seek(0)
        audio = AudioSegment.from_file(audio_file, format="mp3")
        audio = audio.set_frame_rate(16000).set_channels(1)

        # Convert audio to float32 NumPy array
        audio_array = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0

        # Process input
        audio_inputs = processor(audios=audio_array, sampling_rate=16000, return_tensors="pt")
        audio_inputs = {key: val.to(device) for key, val in audio_inputs.items()}  # Ensure tensors are on the correct device
        start_time = time.time()
        # Generate translation
        output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)

        # Extract token IDs from the generated output
        token_ids = output_tokens.sequences
        # Decode token IDs to text
        translated_text_from_audio = processor.batch_decode(token_ids, skip_special_tokens=True)[0]
        end_time = time.time()
        return translated_text_from_audio, end_time - start_time

    except Exception as e:
        return f"Error during audio translation: {e}"


# Function to process text and translate it using SeamlessM4T
def process_text_seamless(text):
    try:
        # Detect language
        language = detect(text)

        # Convert text to speech and save as an in-memory buffer
        tts = gTTS(text=text, lang=language)
        audio_buffer = io.BytesIO()
        tts.write_to_fp(audio_buffer)
        audio_buffer.seek(0)

        # Translate the audio
        translated_text, model_time = seamless_model(audio_buffer)
        return translated_text, model_time

    except Exception as e:
        return f"Error during text processing: {e}"

In [ ]:
output_file = "seamless-m4t-v2-large-results.csv"
existing_df = pd.read_csv(output_file) if os.path.exists(output_file) else None


for language, text in input_text:
    start_time = time.time()
    translated_text, model_time = process_text_seamless(str(text))
    end_time = time.time()

    # Append a new entry to the results DataFrame
    existing_df = update_results(  # Update existing_df with the latest DataFrame
        results_df=existing_df,
        output_path=output_file,
        model="seamless-m4t-v2-large",
        language=language,
        input_text=text,
        translated_text=translated_text,
        model_time=model_time,
        time=end_time - start_time 
    )

print("seamless-m4t-v2-large done")

### Text Processing models

In [ ]:
# Load EuroLLM models
model_path_1b = r"model\eurollm-1.7b-instruct-q4_k_m.gguf"
llm_1b = Llama(model_path=model_path_1b, n_ctx=1024)

# Function to process text using EuroLLM
def process_text_EuroLLM_1B(text, llm):
    input_prompt = f"""
    <|im_start|>system
    <|im_end|>
    <|im_start|>user
    Translate the following text to English:
    Text: {text}
    English: 
    <|im_end|>
    <|im_start|>assistant
    """
    output = llm(input_prompt, max_tokens=1024, temperature=0)
    # print('\n\nOutput: ',output,'\n\n')
    full_response = output.get("choices", [{}])[0].get("text", "").strip()
    # print('\n\nResponse: ',full_response,'\n\n')
    marker = "English:"
    if marker in full_response:
        translated_text = full_response.split(marker)[-1].strip()
        return translated_text if translated_text else "No valid translation provided."
    return "No valid translation provided."


In [ ]:
output_file = "Eurollm-1.7b-results.csv"
existing_df = pd.read_csv(output_file) if os.path.exists(output_file) else None


for language, text in input_text:
    start_time = time.time()
    translated_text = process_text_EuroLLM_1B(str(text), llm_1b)
    end_time = time.time()

    # Append a new entry to the results DataFrame
    existing_df = update_results(
        results_df=existing_df,
        output_path=output_file,
        model="Eurollm-1.7b",
        language=language,
        input_text=text,
        translated_text=translated_text,
        model_time=None,
        time=end_time - start_time 
    )

print("Eurollm-1.7b updated_df")

In [9]:
# Load EuroLLM models
model_path_9b = r"model\EuroLLM-9B-Instruct-Q4_K_M.gguf"
llm_9b = Llama(model_path=model_path_9b, n_ctx=1024)

# Function to process text using EuroLLM
def process_text_EuroLLM_9B(text, llm):
    input_prompt = f"""
    <|im_start|>system
    <|im_end|>
    <|im_start|>user
    Translate the following text to English:
    Text: {text}
    English: 
    <|im_end|>
    <|im_start|>assistant
    """
    output = llm(input_prompt, max_tokens=1024, temperature=0)
    # print('\n\nOutput: ',output,'\n\n')
    full_response = output.get("choices", [{}])[0].get("text", "").strip()
    # print('\n\nResponse: ',full_response,'\n\n')
    return full_response


llama_model_loader: loaded meta data with 41 key-value pairs and 381 tensors from model\EuroLLM-9B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = EuroLLM 9B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = EuroLLM
llama_model_loader: - kv   5:                         general.size_label str              = 9B
llama_model_loader: - kv   6:                            general.license str              = apache-2.0
llama_model_loader: - kv   7:                   

In [10]:
output_file = "EuroLLM-9B-results.csv"
existing_df = pd.read_csv(output_file) if os.path.exists(output_file) else None



for language, text in input_text:
    start_time = time.time()
    translated_text = process_text_EuroLLM_9B(str(text), llm_9b)
    end_time = time.time()

    # Append a new entry to the results DataFrame
    existing_df = update_results(
        results_df=existing_df,
        output_path=output_file,
        model="EuroLLM-9B",
        language=language,
        input_text=text,
        translated_text=translated_text,
        model_time=None,
        time=end_time - start_time 
    )

print("EuroLLM-9B-results updated_df")

llama_perf_context_print:        load time =   19564.72 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    50 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     9 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   21663.75 ms /    59 tokens
C:\Users\moham\AppData\Local\Temp\ipykernel_10952\1039760007.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_entry], ignore_index=True)
Llama.generate: 26 prefix-match hit, remaining 29 prompt tokens to eval
llama_perf_context_print:        load time =   19564.72 ms
llama_perf_context_print: prompt eval time =       0.

EuroLLM-9B-results updated_df
